# Sequential Rule Mining Using SPMF

This practical note introduces you to sequential rule mining using SPMF. SPMF is a Java library implemented by Philippe Fournier Viger containing 150 pattern mining algorithms, such as frequent pattern mining, association rules, frequent sequences and of course, sequential rule mining. We will use SPMF as there are no mature library in Python that implement sequential rule mining to the extend of SPMF.

You can find information about SPMF here http://www.philippe-fournier-viger.com/spmf/index.php and you can download SPMF here http://www.philippe-fournier-viger.com/spmf/index.php?link=download.php. Put the `spmf.jar` file into the same directory with this Jupyter notebook.

As SPMF is implemented in Java, you need to install Java to run the `.jar` file. We will not discuss how to do that, there are many tutorials on the Internet. To check if you have installed Java correctly for this practical, open your terminal/CMD and type `java`.

To demonstrate how to use SPMF through Python, we will use the `bank.csv` dataset used in the past practical. Import it through pandas.

In [1]:
import pandas as pd
df = pd.read_csv('bank.csv')

df.head(10)

,ACCOUNT,SERVICE,VISIT
0,500026,CKING,1
1,500026,SVG,2
2,500026,ATM,3
3,500026,ATM,4
4,500075,CKING,1
5,500075,MMDA,2
6,500075,SVG,3
7,500075,ATM,4
8,500075,TRUST,5
9,500075,TRUST,6


Different from association mining, to get sequential rules, there must be information of order of the target. In this dataset, the order is given in `VISIT` column. As this dataset is already ordered by `VISIT`, simple group and apply list will produce sequences in order. In other datasets, you will have to perform some preprocessing to ensure this ordering is correct.

In [2]:
transactions = df.groupby(['ACCOUNT'])['SERVICE'].apply(list)
sequences = transactions.values.tolist()

# show the first 5 sequences
print(sequences[:5])

[['CKING', 'SVG', 'ATM', 'ATM'], ['CKING', 'MMDA', 'SVG', 'ATM', 'TRUST', 'TRUST'], ['CKING', 'SVG', 'IRA', 'ATM', 'ATM'], ['CKING', 'SVG', 'CKCRD', 'CKCRD'], ['CKING', 'SVG', 'CKCRD', 'CKCRD']]


Once you have sequences ordered correctly, you could simply run the following function to get sequential rules and their respective support and confidence. In general, the function below will write your sequences into a file called `seq_rule_input.txt` in SPMF accepted format, run SPMF to generate sequential rules, read the output file and return a Pandas dataframe. Detailed comments is provided in the function source code.

In [9]:
from collections import defaultdict
import subprocess
import re

''' Uses SPMF to find association rules in supplied transactions '''
def get_association_rules(sequences, min_sup, min_conf):
    # step 1: create required input for SPMF
    
    # prepare a dict to uniquely assign each item in the transactions to an int ID
    item_dict = defaultdict(int)
    output_dict = defaultdict(str)
    item_id = 1
    
    # write your sequences in SPMF format
    with open('seq_rule_input.txt', 'w+') as f:
        for sequence in sequences:
            z = []
            for itemset in sequence:
                # if there are multiple items in one itemset
                if isinstance(itemset, list):
                    for item in itemset:
                        if item not in item_dict:
                            item_dict[item] = item_id
                            item_id += 1

                        z.append(item_dict[item])
                else:
                    if itemset not in item_dict:
                        item_dict[itemset] = item_id
                        output_dict[str(item_id)] = itemset
                        item_id += 1
                    z.append(item_dict[itemset])
                    
                # end of itemset
                z.append(-1)
            
            # end of a sequence
            z.append(-2)
            f.write(' '.join([str(x) for x in z]))
            f.write('\n')
    
    # run SPMF with supplied parameters
    supp_param = '{}%'.format(int(min_sup * 100))
    conf_param = '{}%'.format(int(min_conf * 100))
    subprocess.call(['java', '-jar', 'spmf.jar', 'run', 'RuleGrowth', 'seq_rule_input.txt', 'seq_rule_output.txt', '10%', '10%'], shell=True)
    
    # read back the output rules
    outputs = open('seq_rule_output.txt', 'r').read().strip().split('\n')
    output_rules = []
    for rule in outputs:
        left, right, sup, conf = re.search(pattern=r'([0-9\,]+) ==> ([0-9\,]+) #SUP: ([0-9]+) #CONF: ([0-9\.]+)', string=rule).groups()
        sup = int(sup) / len(sequences)
        conf = float(conf)
        output_rules.append([[output_dict[x] for x in left.split(',')], [output_dict[x] for x in right.split(',')], sup, conf])
    
    # return pandas DataFrame
    return pd.DataFrame(output_rules, columns = ['Left_rule', 'Right_rule', 'Support', 'Confidence'])

You can run the function on sequences from the `bank.csv` dataset using the command below. In here, we are using `min_supp` of 0.1 and `min_conf` of 0.1.

In [12]:
get_association_rules(sequences, 0.1, 0.1)

,Left_rule,Right_rule,Support,Confidence
0,[CKING],[SVG],0.024402,0.343915
1,"[CKING, ATM]",[SVG],0.003629,0.381579
2,"[CKING, ATM, MMDA]",[SVG],0.002878,0.348485
3,"[CKING, ATM, MMDA, TRUST]",[SVG],0.002127,0.404762
4,"[CKING, ATM, MMDA, TRUST, ]",[SVG],0.001377,0.846154
5,"[CKING, ATM, MMDA, TRUST, IRA]",[SVG],0.002127,0.404762
6,"[CKING, ATM, MMDA, TRUST, IRA, ]",[SVG],0.001377,0.846154
7,"[CKING, ATM, MMDA, TRUST, IRA, CKCRD]",[SVG],0.002127,0.404762
8,"[CKING, ATM, MMDA, TRUST, IRA, CKCRD, ]",[SVG],0.001377,0.846154
9,"[CKING, ATM, MMDA, TRUST, IRA, CKCRD, HMEQLC]",[SVG],0.001126,0.692308


In [ ]:
java -jar D:\Documents\645Ass2\IFN_645\CaseStudy_2\spmf.jar run RuleGrowth D:\Documents\645Ass2\IFN_645\CaseStudy_2\seq_rule_input.txt 0.01 0.01

The first rule is `CKING` => `SVG` for 0.54 support and 0.63 confidence. This is a strong rule, as it implies 54% of customers bought `SVG` after `CKING` (support) and if customer bought `CKING`, the probability of them buying `SVG` after is 63%.

You can now use the function to perform web log mining in the assignment.